In [26]:
import pandas as pd 

input = pd.read_csv("archive/Amazon-Products.csv")

In [27]:
import random
import numpy as np 

df = input.copy(deep=True)
split_by_paren = df['name'].str.split(pat='(', n=1, expand=True) # split name into short name and description
split_by_paren[1] = split_by_paren[1].str[:-1] # remove last )
df['name'] = split_by_paren[0]
df.insert(2, 'description', split_by_paren[1])
df.drop(columns=["discount_price", "link"], inplace=True)
df['actual_price'] = (df['actual_price'].str.strip('₹').str.replace(',', '')\
    .astype(float) * 0.012).fillna(random.uniform(0, 100)).round(decimals=2) # convert to USD
df['no_of_ratings'] = df['no_of_ratings'].str.replace(',', '').str.extract(r'(\d+)').fillna('0').astype(int)
df.drop(df[df.ratings.str.contains(pat='₹', regex=False).astype(bool).fillna(False)].index, inplace=True)
df['ratings'] = df['ratings'].str.extract(r'(\d+.\d+)').astype(float).fillna(0.)
df.drop(columns="Unnamed: 0", inplace=True)
# df.rename(columns={'Unnamed: 0':'ID'}, inplace=True)
df.drop_duplicates(subset="name", inplace=True)
df.drop(df[df['name']==""].index, inplace=True)

df['rand'] = np.random.randint(0, 10, df.shape[0])
df['available'] = np.where(df['rand'] >= 1, True, False) # 90% available
df.drop(columns='rand', inplace=True)

df.drop(df[df['actual_price'] > 2000].index, inplace=True)
df = df[['name', 'actual_price', 'available', 'description', 'main_category', 'sub_category', 'ratings', 'no_of_ratings', 'image']]

df.set_index(np.arange(0, len(df)), inplace=True)
df

,name,actual_price,available,description,main_category,sub_category,ratings,no_of_ratings,image
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac,707.88,True,"5 In 1 Convertible, Copper, Anti-Viral + Pm 2....",appliances,Air Conditioners,4.2,2255,https://m.media-amazon.com/images/I/31UISB90sY...
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC,911.88,True,"Copper, Super Convertible 6-in-1 Cooling, HD F...",appliances,Air Conditioners,4.2,2948,https://m.media-amazon.com/images/I/51JFb7FctD...
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac,743.88,True,"Copper, Super Convertible 6-In-1 Cooling, Hd F...",appliances,Air Conditioners,4.2,1206,https://m.media-amazon.com/images/I/51JFb7FctD...
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC,827.88,True,"Copper, Super Convertible 6-in-1 Cooling, HD F...",appliances,Air Conditioners,4.0,69,https://m.media-amazon.com/images/I/51JFb7FctD...
4,Carrier 1.5 Ton 3 Star Inverter Split AC,813.48,True,"Copper,ESTER Dxi, 4-in-1 Flexicool Inverter, 2...",appliances,Air Conditioners,4.1,630,https://m.media-amazon.com/images/I/41lrtqXPiW...
...,...,...,...,...,...,...,...,...,...
254851,BE SAVAGE Pink I Purple You Hoodie Suitable fo...,11.99,False,None,sports & fitness,Yoga,4.6,6,https://m.media-amazon.com/images/I/61voQ68VuT...
254852,Adidas Regular Fit Men's Track Tops,55.19,True,None,sports & fitness,Yoga,3.2,9,https://m.media-amazon.com/images/I/71tHAR9pIY...
254853,Redwolf Noice Toit Smort - Hoodie,23.99,False,Black,sports & fitness,Yoga,2.0,2,https://m.media-amazon.com/images/I/41pKrMZ5lQ...
254854,Redwolf Schrute Farms B&B - Hoodie,23.99,True,Navy Blue,sports & fitness,Yoga,4.0,1,https://m.media-amazon.com/images/I/41n9u+zNSc...


In [28]:
df.to_csv("Amazon_Products.csv", header=False)